In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim 
import torchvision
import sklearn.metrics as sm 
from PIL import Image
from PIL import ImageOps
from glob import glob
import time

In [0]:
files=glob('png/*/*',recursive=True)

for file in sorted(files):
    img=Image.open(file)
    img=img.resize((180,180),Image.BICUBIC)
    img=ImageOps.invert(img)
    img=np.array(img)
    img[img>0]=255
    img=Image.fromarray(img)
    img.save(file)

In [0]:
class Model(nn.Module):

    def __init__(self):
        super(Model,self).__init__()
        self.conv1=nn.Conv2d(1,64,7,2,0)
        self.conv2=nn.Conv2d(64,128,5,2,2)
        self.conv3=nn.Conv2d(128,256,3,1,1)
        self.conv4=nn.Conv2d(256,512,3,1,0)
        self.conv5=nn.Conv2d(512,4096,5,1,0)
        self.conv6=nn.Conv2d(4096,250,1,1,0)
        self.maxpool=nn.MaxPool2d(3,2,0)
        self.dropout=nn.Dropout()

    def forward(self,x):
        x = self.conv1(x)
        x = f.relu(x)
        x = self.maxpool(x)

        x = self.conv2(x)
        x = f.relu(x)
        x = a = self.maxpool(x)
        
        x = self.conv3(x)
        x = f.relu(x)

        x = self.conv4(x)
        x = f.relu(x)
        x = b = self.maxpool(x)

        x = self.conv5(x)
        x = c = f.relu(x)
        
        x = self.dropout(x)
        
        x = self.conv6(x)
        x = x.view(-1,250)

        return (x,a,b,c)

In [0]:
def get_data_loader(root,batch_size):
    img_transform=torchvision.transforms.Compose([
            torchvision.transforms.Pad(22),
            torchvision.transforms.RandomAffine(degrees=35),
            torchvision.transforms.ToTensor()
        ])
    dataset=torchvision.datasets.ImageFolder(root,img_transform)
    loader=torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    return loader

In [0]:
train_loader=get_data_loader('./png/',64)
print(len(train_loader))
for data in train_loader:
    x,y=data
    print(x[0][0].shape)
    x=x[:,0].unsqueeze(1)
    img=Image.fromarray(x[0][0].numpy())
    img.show()
    print(x.shape)
    break

In [0]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:',device)

model='model_1.pt'
logger = open(model[:-3]+'_train.txt','w')

Net=Model().to(device=device)

lossfunc=nn.CrossEntropyLoss()
optimizer=optim.Adam(Net.parameters())

batch_size=64
epochs=2
printfreq=100

train_loader=get_data_loader('./png/',batch_size)

logger.write("Device : %s\n\n"%(device))
logger.write(str(Net)+'\n\n')
logger.write("Optimizer : ADAM\n\n")
logger.write("Batch Size : %d\n\n"%(batch_size))
logger.write("Epochs : %d\n\n"%(epochs))

print("\nTraining...\n")

start_time=time.time()

for ep in range(epochs):
    
    running_loss=0.0
    train_loss=0.0
    test_loss=0.0

    for i,data in enumerate(train_loader):
        x,y = data
        
        x=x[:,0].unsqueeze(1)

        x,y = x.to(device=device),y.to(device=device)
        
        optimizer.zero_grad()

        output = Net(x)

        loss = lossfunc(output[0],y)
        
        running_loss+=loss.item()
        train_loss+=loss.item()

        loss.backward()
        optimizer.step()

        if (i+1)%printfreq==0:
            print("**** Epoch %d ****\nBatch: %d\tRunning Loss: %.4f\n"%(ep+1,i+1,running_loss))
            running_loss=0.0
        
    logger.write("**** Epoch %d ****\nTrain Loss: %.4f\nTest Loss: %.4f\n\n"%((ep+1),train_loss,test_loss))

end_time=time.time()

print("Training Complete. Time Taken: %.4f\n"%(end_time-start_time))

torch.save(Net,model)

In [0]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model='model_1.pt'
Net=torch.load(model,device)
logger = open(model[:-3]+'_performance.txt','w')

features=torch.load('features.pt').to(device=device)
labels=np.load('labels.npy')

prediction=Net(features).detach().cpu().round()

logger.write(sm.classification_report(labels,prediction))
